## 데이터 및 라이브러리 로드

In [ ]:
import os
import random
import pickle
import numpy as np
import pandas as pd
import gc
import tensorflow as tf
from custom_metric import *
import tensorflow_addons as tfa

SEED = 30
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
%%time
# 약 20초 걸림
with open('D:/inputs/24/train50.pickle', 'rb') as f:
    train = pickle.load(f)
with open('D:/inputs/24/train_y50.pickle', 'rb') as f:
    train_y = pickle.load(f)
    train_y = train_y.reshape(train_y.shape[0], 40, 40, 1)


with open('D:/inputs/24/test.pickle', 'rb') as f:
    TEST = pickle.load(f)
TEST = TEST[:, :, :, :10]    
# 아웃라이어 수정
# train[1,16, 24, -2] = 163.05731201

## 회전변환

In [ ]:
## 회전변환시 마이너스(-) 하는 경우, 제곱을 해주면 target값과의 상관관계가 높아짐. 테스트해볼것!
# v1_m_h1 = ((train[:, :, :, 0] * np.cos(np.pi / 4) - train[:, :, :, 1] * np.sin(np.pi / 4)) ** 0.5) / 100

In [ ]:
v1_m_h1 = train[:, :, :, 0] * np.cos(np.pi / 4) - train[:, :, :, 1] * np.sin(np.pi / 4)  
v1_p_h1 = train[:, :, :, 0] * np.cos(np.pi / 4) + train[:, :, :, 1] * np.sin(np.pi / 4)  
train[:, :, :, 0] = v1_m_h1
train[:, :, :, 1] = v1_p_h1 
del v1_m_h1
del v1_p_h1

v2_m_h2 = train[:, :, :, 2] * np.cos(np.pi / 4) - train[:, :, :, 3] * np.sin(np.pi / 4)
v2_p_h2 = train[:, :, :, 2] * np.cos(np.pi / 4) + train[:, :, :, 3] * np.sin(np.pi / 4)
train[:, :, :, 2] = v2_m_h2
train[:, :, :, 3] = v2_p_h2 
del v2_m_h2
del v2_p_h2


In [ ]:
v1_m_h1 = TEST[:, :, :, 0] * np.cos(np.pi / 4) - TEST[:, :, :, 1] * np.sin(np.pi / 4)  
v1_p_h1 = TEST[:, :, :, 0] * np.cos(np.pi / 4) + TEST[:, :, :, 1] * np.sin(np.pi / 4)  
TEST[:, :, :, 0] = v1_m_h1
TEST[:, :, :, 1] = v1_p_h1 
del v1_m_h1
del v1_p_h1

v2_m_h2 = TEST[:, :, :, 2] * np.cos(np.pi / 4) - TEST[:, :, :, 3] * np.sin(np.pi / 4)
v2_p_h2 = TEST[:, :, :, 2] * np.cos(np.pi / 4) + TEST[:, :, :, 3] * np.sin(np.pi / 4)
TEST[:, :, :, 0] = v2_m_h2
TEST[:, :, :, 1] = v2_p_h2 
del v2_m_h2
del v2_p_h2

In [ ]:
v4_p_h4_30 = train[:, :, :, 5] * np.cos(np.pi / 6) + train[:, :, :, 6] * np.sin(np.pi / 6)
v4_m_h4_30 = train[:, :, :, 5] * np.cos(np.pi / 6) - train[:, :, :, 6] * np.sin(np.pi / 6)
train[:, :, :, 5] = v4_p_h4_30
train[:, :, :, 6] = v4_m_h4_30

v5_p_h5_30 = train[:, :, :, 7] * np.cos(np.pi / 6) + train[:, :, :, 8] * np.sin(np.pi / 6)
v5_m_h5_30 = train[:, :, :, 7] * np.cos(np.pi / 6) - train[:, :, :, 8] * np.sin(np.pi / 6)
train[:, :, :, 7] = v5_p_h5_30
train[:, :, :, 8] = v5_m_h5_30

del v4_p_h4_30
del v4_m_h4_30
del v5_p_h5_30
del v5_m_h5_30

In [ ]:
v4_p_h4_30 = TEST[:, :, :, 5] * np.cos(np.pi / 6) + TEST[:, :, :, 6] * np.sin(np.pi / 6)
v4_m_h4_30 = TEST[:, :, :, 5] * np.cos(np.pi / 6) - TEST[:, :, :, 6] * np.sin(np.pi / 6)
TEST[:, :, :, 5] = v4_p_h4_30
TEST[:, :, :, 6] = v4_m_h4_30

v5_p_h5_30 = TEST[:, :, :, 7] * np.cos(np.pi / 6) + TEST[:, :, :, 8] * np.sin(np.pi / 6)
v5_m_h5_30 = TEST[:, :, :, 7] * np.cos(np.pi / 6) - TEST[:, :, :, 8] * np.sin(np.pi / 6)
TEST[:, :, :, 7] = v5_p_h5_30
TEST[:, :, :, 8] = v5_m_h5_30

del v4_p_h4_30
del v4_m_h4_30
del v5_p_h5_30
del v5_m_h5_30

In [ ]:
# 4개의 경도, 위도 정보 삭제함.
train = train[:, :, :, :10]

In [ ]:
# 4개의 경도, 위도 정보 삭제함.
TEST = TEST[:, :, :, :10]

In [ ]:
train.shape

In [ ]:
TEST.shape

In [ ]:
train = np.concatenate([train, train_y], -1)

In [ ]:
%%time

train1 = np.rot90(train, 1, (1,2))
train2 = np.rot90(train, 2, (1,2))
train3 = np.rot90(train, 3, (1,2))
train_lr = np.fliplr(train)
# train_ud = np.flipud(train)

train = np.vstack([train, train1])
del train1

train = np.vstack([train, train2])
del train2

train = np.vstack([train, train3])
del train3

train = np.vstack([train, train_lr])
del train_lr

# train = np.vstack([train, train_ud])
# del train_ud

In [ ]:
train_y = train[:, :, :, -1].copy()
train_y = train_y.reshape(train_y.shape[0], train_y.shape[1], train_y.shape[2], 1)

In [ ]:
train = train[:,:,:,:-1]

In [ ]:
from sklearn.model_selection import train_test_split
train, test, train_y, test_y = train_test_split(train, train_y, test_size=0.025, random_state=SEED)

In [ ]:
gc.collect()

## 9번 channel 가공 - 카테고리형 피쳐

In [ ]:
# [  0., 100., 101., 102., 103., 105., 106., 107., 108., 109., 110.,
#        111., 112., 113., 114., 115., 117., 118., 119., 121., 122., 123.,
#        199., 200., 201., 202., 203., 205., 206., 207., 208., 210., 211.,
#        212., 213., 214., 215., 217., 218., 219., 221., 222., 223., 299.,
#        300., 301., 302., 303., 305., 306., 307., 308., 310., 311., 312.,
#        313., 314., 315., 317., 318., 319., 321., 322.]
# train = np.delete(train, 9, axis=3)
# test = np.delete(test, 9, axis=3)

In [ ]:
# from features import terrain

# train, test = terrain(train, test)

## all_np로 결합 후 scaling

In [ ]:
## ss 약 5분 걸림
# from preprocessing import scaling

# train, test = scaling(train, test, 'ss') # 3번째 인자 'rs', 'ms', 'ss' 


## 모델 선언 - Unet, Resnet

In [ ]:
from models import build_model, build_model2
from tensorflow.keras.layers import Input, Conv2D, Add, BatchNormalization, concatenate
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

In [ ]:
input_layer = Input((40, 40, train.shape[3]))
output_layer = build_model(input_layer, 32)

# sgd = tf.keras.optimizers.SGD(0.01)
# stocastic_avg_opt = tfa.optimizers.SWA(sgd)

# ad = tf.keras.optimizers.Adam(0.01)
# stocastic_avg_opt = tfa.optimizers.SWA(ad)
# model.compile(loss="mae", optimizer=stocastic_avg_opt, metrics=["mae"])
#


# model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])

scores = []

In [ ]:
def resnet_model():
    inputs=Input(x_train.shape[1:])
    
    bn=BatchNormalization()(inputs)
    conv0=Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    bn=BatchNormalization()(conv0)
    conv=Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([conv0, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    conv=Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([concat, conv], axis=3)
        
    for i in range(5):
        bn=BatchNormalization()(concat)
        conv=Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat=concatenate([concat, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    outputs=Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    model=Model(inputs=inputs, outputs=outputs)
    
    return model

In [ ]:
model = tf.keras.models.load_model('epochs_8.h5')
model.summary()

In [ ]:
indexes = []
for train_idx in k_fold.split(train):
    indexes.append(train_idx)

In [ ]:
# 인덱스 로드해서 잘 복원되는지 확인해볼것.
# import pickle
# with open('submissions/22_23_kfold_index.pickle', 'wb') as f:
#     pickle.dump(indexes, f)

In [ ]:
indexes[3]

## k-fold 모델 훈련 및 모델 저장

In [ ]:
from sklearn.model_selection import KFold

FOLD = 4
k_fold = KFold(n_splits=FOLD, shuffle=True, random_state=SEED)

EPOCHS = 60
history = []
scores = []

result = 0

In [ ]:
model_number = 0
for train_idx, val_idx in k_fold.split(train, train_y):

    x_train, y_train = train[train_idx], train_y[train_idx]
    x_val, y_val = train[val_idx], train_y[val_idx]
    
#     model = Model(input_layer, output_layer)
    model = resnet_model()
    model.compile(loss="mae", optimizer="adam", metrics=["mae"])
    
    es = EarlyStopping(patience=6, verbose=1)
    mc = ModelCheckpoint(f'best_{model_number}.h5', save_best_only=True, verbose=1)
    rlp = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.8)
    csv_logger = CSVLogger(f'training_{model_number}.csv')
    
    model_ = model.fit(x_train, np.log(y_train+1), epochs = EPOCHS, validation_data=(x_val, np.log(y_val+1)), verbose=1, batch_size = 64, callbacks = [es, mc, rlp, csv_logger])
        
        
    score = maeOverFscore(y_val, np.exp(model.predict(x_val))-1)
    print('mae / fscore: ' + str(score) + '\n')
    scores.append(score)
    print(scores)
    
    history.append(model_)
    model.load_weights(f'best_{model_number}.h5')
    result += model.predict(TEST) / FOLD
    model_number += 1

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10,5))
plt.plot(history[0].history['val_loss'])

## 자체 test셋으로 검증
리더보드와 매우 비슷

In [ ]:
# with open('inputs/my_test_set.pickle', 'rb') as f:
#     test, test_y = pickle.load(f)

In [ ]:
model_score = 0
tmp = 0
for i in range(4):
    model = resnet_model()
    model.compile(loss="mae", optimizer="adam", metrics=["mae"])
    model.load_weights(f'best_{i}.h5')
    tmp += model.predict(test) / 4
    
model_score = maeOverFscore(test_y, tmp)
model_score

## submission 생성

In [ ]:
result = 0
for i in range(4):
    model = resnet_model()
    model.compile(loss="mae", optimizer="adam", metrics=["mae"])
    model.load_weights(f'best_{i}.h5')
    result += model.predict(TEST)
    

In [ ]:
submission = pd.read_csv('inputs/sample_submission.csv')
submission.iloc[:,1:] = result.reshape(-1, 1600)
submission.to_csv('kfold_resnet_4_rot.csv', index = False)

## 앙상블

In [ ]:
tmp1 = pd.read_csv('22_unet_1.csv')
tmp2 = pd.read_csv('23_kfold_resnet_4.csv')

tmp1.iloc[:, 1:] = (tmp1.iloc[:, 1:] * 0.5) + (tmp2.iloc[:, 1:] * 0.5)

tmp1.to_csv('ensemble_22_23.csv', index = False)

## 해야할 일

1. ~~스케일링(Scaling) - Robust, log, Standard Scaler  ~~  
스케일링 방법을 다르게 해야하나. 하나의 사진을 기준으로 처리해야 할지 아니면 1번픽셀, 2번픽셀... 1600픽셀 끼리 해야하나
2. ~~위도 경도 피쳐를 가지고 위도+경도, 위도-경도 피쳐 만들기(45도 회전변환)~~: 30, 60도 변환 추가
3. 현재 custom metric이 작동하지 않으므로, 이 또한 수정해야함.
4. tree모델 만들고, 앙상블
5. ~~target 값에 log 취하기~~: 점수 안오름
6. 순서가 바뀌긴 했으나, EDA 작업하자.
7. 강수량 타일이 10개 미만인 것의 평균, 20개미만의 평균.... 50개 미만의 평균... 알아보자
8. ~~augumentation 사용해서 강수량이 일정수준 이상인 사진 augementation 하자~~ - 다만, regression인 만큼 가장자리 어떻게 처리할지 생각해보자. - 정사각형 사진이기에 90도 돌린사진, 180도 돌린사진, 270도 돌린사진 이렇게만 해도 데이터가 증식될거같음 -  
**그런데!!!!!** 생각해봐야할것이.....  
위도, 경도 등 여러 요소가 존재할텐데 돌려버리면 원래는 비 안오는거를 비온다고 볼 수도 있을 듯 싶다. 쉽게 예를 들면 I 모양일때는 비가 오고, ㅡ 모양일때는 비가안오는데, 돌리는 바람에 안오는걸 온다고 할 수도 있을 것 같다. - 이건 테스트해볼수밖에 없을 듯 싶다.  
돌릴수있는방법도 여러가지이다. 90도씩회전하기, 수직축기준으로 전환, 수평축기준으로 전환 등 - 캐글에서는 수직축, 수평축으로 전환해서 augmentation 썼다고 함.  위성이 도는 방향을 보면, 270도 돌리는 것도 괜찮을듯
9. pretrained 모델 활용방안 생각하기 - 최소 사진크기가 존재하는데 우리는 40x40이라 작은편.. 이걸 40x40을 이어 붙여서 200x200으로 만들어서 넣으면 될거같은데 시도해보자.
10. 9번채널 카테고리 변수 원핫인코딩
11. Unet 모델과 FPN 모델 앙상블해서 제출하자
12. loss='mse', metric=['mse', 'mae'] 이렇게 해보자
13. U_res_net 구현하기
14. 회전변환한 걸로 U_net 돌려서 제출하기
15. off nadir viewing angle, earth incidence angle

| 리더보드(5부터) | mae / fscore | mae | 비고 |
|---|:---:|---:|---:|
| 1.93737 | 1.8744 | 0.2451 |  |
| 1.75177 | 1.7016 | 0.2384 |  |
| 1.61793 | 1.4234 | 0.0832 | 리더보드랑 metric 점수랑 차이 많이 남. 해결해야함. |